 ### 아래 내용은 intel AI for Youth Program 내용을 참고하여
<font color="blue"> Brain AI와 Brain AI Coach Network에서 개발한 내용입니다. 상업적 사용은 불가하며,<br>
학교에서 학생들 교육활동에 자유롭게 사용가능합니다. </font>

# Project Title: BAI_Sorter 인공지능 자동 분류기 만들기
구글 티처블 머신에서 개발한 인공지능 모델을 불러와 <br>
마이크로비트와 BAI 마이크로비트 확장보드, 서보모터로 작동하는 인공지능 자동 분류기를 개발하여 봅시다.

관련 영상 링크입니다. <a href="https://vimeo.com/483847682"> 인공지능 자동 분류기 프로젝트 영상 </a>

In [2]:
import tensorflow as tf
print(tf.__version__)

2.6.0


In [3]:
# 필요 라이브러리 불러오기
import numpy as np
import cv2
import time

import serial
import serial.tools.list_ports

import tensorflow.keras
from tensorflow.keras.models import model_from_json

In [4]:
# USB 시리얼 포트 자동 찾기
ports = serial.tools.list_ports.comports()
com = ''
for port, desc, hwid in sorted(ports):        
    if 'USB' in desc:
        com = port
if com != '':
    print('Micro:bit detected: ', com)  
else:
    print('Please connect your microbit to this PC via USB') 

Micro:bit detected:  COM5


In [5]:
# 레이블 불러오기
labels = []
file = open("model/labels.txt", "r")
for x in file:
    labels.append(x.rstrip('\n'))
print('labels = ', labels)
file.close()

#인공지능 모델 불러오기
model = tensorflow.keras.models.load_model('model/keras_model.h5', compile = False)

labels =  ['0 Red', '1 orange', '2 Empty']


In [6]:
# 시리얼 통신으로 명령 보내기
def SerialSendCommand(cmd):    
    cmd = str(cmd) 
    cmd  = cmd + '\n'
    cmd = str.encode(cmd) 
    ser.write(cmd)  

In [7]:
# 새로운 이미지 데이터 읽어와 분류 예측하기
def ReadPicture(cropped, model):   
    input_width = 224
    input_height = 224

    resized_image = cv2.resize(cropped, (input_width, input_height))
    imgRGB =cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    image_array = np.asarray(imgRGB)
    
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    data = normalized_image_array.reshape(1, input_height, input_width, 3)

    prediction = model.predict(data)
    
    class_id = np.argmax(prediction)
    score = prediction[0][class_id]
    
    return class_id, score

### Teachable Machine Guide 
- Create the array of the right shape to feed into the keras model
- The 'length' or number of images you can put into the array is
- determined by the first position in the shape tuple, in this case 1.
- data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

In [8]:
# 메인 함수
def Main():
        
    MODE_PAUSE = -1 
    MODE_START = 0
    mode_status = MODE_PAUSE

    color  = (0, 0, 255)
    cmd = 90
        
    tic = time.time()
    text = "Press 'p' to start"
  
    video_capture = cv2.VideoCapture(0)
    height = video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT )
    width = video_capture.get(cv2.CAP_PROP_FRAME_WIDTH )

    y1 = int(height - (height * top))
    y2 = int(height - (height * bottom))
    square = (y2-y1)/2
    x1 = int(width/2 - square)
    x2 = int(width/2 + square)
    
    print("Press [q] to quit") 
    print("Press [p] to start/pause")
    
    while(True):

        grabbed, frame = video_capture.read()
        cropped = frame[y1:y2, x1:x2]
 
        if mode_status == MODE_START:
            if time.time() > tic:      
                tic = time.time() + 1
                if cmd == 90:    
                    class_id, score = ReadPicture(cropped, model)   
                    text = labels[class_id][1:] + ' ' + str(int(score*100)) + '%' 
                    if class_id == 0 and score > .75: 
                        cmd = 165
                    elif class_id == 1 and score > .75:
                        cmd = 15                       
                    elif class_id == 2:
                        pass                
                elif cmd != 90:
                    cmd = 90
                SerialSendCommand(cmd)
                             
        cv2.putText(frame, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
        cv2.imshow("Frame", frame) 
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord("q"):
            break

        elif key == ord('p'):
            if mode_status != MODE_PAUSE:
                mode_status = MODE_PAUSE       
                color = (0, 0, 255)
                text = "Press 'p' to start"
                SerialSendCommand(90)
            elif mode_status == MODE_PAUSE:
                mode_status = MODE_START
                color = (0, 255, 0)
                text = "Press 'p' to pause"

    video_capture.release()
    ser.close()
    cv2.destroyAllWindows()

In [ ]:
#설정 값 변경 및 프로그램 실행
top = .60
bottom = .25
ser = serial.Serial(com, 115200, timeout=0,parity=serial.PARITY_NONE, rtscts=0)

if __name__ == '__main__': 
    Main()

Press [q] to quit
Press [p] to start/pause
